In [1]:
import os
import sys
import warnings
import random
import copy
import pickle
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('../common')
import data_io_utils
import paths
import constants
import utils

import A006_common


%reload_ext autoreload
%autoreload 2

In [2]:
agg_gfp_file = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 
        'A051a_mlpe-gfp-pilot_Data_Efficiency_Chip_1_GFP_Cloning_Validated.pkl')
agg_blac_file = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 
        'A051a_mlpe-gfp-pilot_Data_Efficiency_Chip_1_Beta-Lactamase_Cloning_Validated.pkl')

data_io_utils.sync_s3_path_to_local(agg_gfp_file, is_single_file=True)
data_io_utils.sync_s3_path_to_local(agg_blac_file, is_single_file=True)

In [3]:
gfp_df = pd.read_pickle(agg_gfp_file)
blac_df = pd.read_pickle(agg_blac_file)

display(gfp_df.head())
display(blac_df.head())

master_df = pd.concat([gfp_df, blac_df], axis=0)
print(master_df.shape)

,id,seq_idx,trajectory_idx,predicted_fitness,ensemble_predicted_fitness,seq,protein,pred_score,lib_aa_seq,constant_where_expected,successfully_clones,error_message,dna_to_order,ligated_gene,ligated_gene_translated,n_mut_rel_wt,oligo_len
0,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,615.0,1516.0,0.835807,"[0.8386957, 0.8833945, 0.6869724, 0.7490394, 0...",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKMTLKF...,GFP,NaN,SGEGEGDATYGKMTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYP...,True,True,None,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...,ATGAGTAAAGGCGAAGAGCTGTTCACTGGTGTAGTCCCGATTCTGG...,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKMTLKF...,2,293
1,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,250.0,922.0,0.831929,"[0.83363, 0.8859961, 0.6875845, 0.74543333, 0....",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,GFP,NaN,SGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVKCFSRYP...,True,True,None,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...,ATGAGTAAAGGCGAAGAGCTGTTCACTGGTGTAGTCCCGATTCTGG...,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,2,293
2,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,250.0,195.0,0.830867,"[0.8327829, 0.87653375, 0.6890327, 0.73599446,...",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,GFP,NaN,SGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYP...,True,True,None,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...,ATGAGTAAAGGCGAAGAGCTGTTCACTGGTGTAGTCCCGATTCTGG...,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,0,293
3,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,354.0,1847.0,0.828640,"[0.82065207, 0.8765222, 0.6913246, 0.747247, 0...",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATMGKLTLKF...,GFP,NaN,SGEGEGDATMGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYP...,True,True,None,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...,ATGAGTAAAGGCGAAGAGCTGTTCACTGGTGTAGTCCCGATTCTGG...,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATMGKLTLKF...,3,293
4,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,933.0,163.0,0.825671,"[0.83166367, 0.8713553, 0.69725585, 0.7542225,...",MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDASYGKMTIKF...,GFP,NaN,SGEGEGDASYGKMTIKFICTTGKLPVPWPTLVTTLSYGVQCFSRYP...,True,True,None,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...,ATGAGTAAAGGCGAAGAGCTGTTCACTGGTGTAGTCCCGATTCTGG...,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDASYGKMTIKF...,5,293


,id,seq_idx,trajectory_idx,predicted_fitness,ensemble_predicted_fitness,seq,protein,pred_score,lib_aa_seq,constant_where_expected,successfully_clones,error_message,dna_to_order,ligated_gene,ligated_gene_translated,n_mut_rel_wt,oligo_len
0,BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e7216...,2595.0,1773.0,0.847448,"[0.7188464, 0.80541486, 0.83650917, 1.0177329,...",MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,BLAC,NaN,ANLLLTSIGGPMELTHFLHNMGDHVTRLDRWEEHLNEAIPNDERDT...,True,True,None,CGCGTCGAGTAGGGAAGACAATGCGGCCAACTTACTTCTGACAAGT...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,7.0,293.0
1,BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e7216...,855.0,2580.0,0.843270,"[0.71885014, 0.805406, 0.8365229, 0.9768362, 0...",MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,BLAC,NaN,ANCLLFTIGGPKELVAFLKNMGDHVTRLDRWNTELNEAIPNDERDA...,True,False,NoValidCodonFoundException(),None,None,None,NaN,NaN
2,BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e7216...,2269.0,1955.0,0.838940,"[0.7188311, 0.8054163, 0.8365104, 1.0334604, 0...",MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,BLAC,NaN,ANLLLTTIGGPKELTAFLHNMGDHVTRLDKWHPELARSIPNDQRDT...,True,True,None,CGCGTCGAGTAGGGAAGACAATGCGGCCAACTTACTTCTGACAACG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,6.0,293.0
3,BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e7216...,2720.0,1554.0,0.838937,"[0.7188321, 0.8054168, 0.8365092, 1.0048122, 0...",MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,BLAC,NaN,ANLLLTTIGGPKELTAFLHNMGDHVTRHDQYKPQLNEAIPNDERDT...,True,True,None,CGCGTCGAGTAGGGAAGACAATGCGGCCAACTTACTTCTGACAACG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,7.0,293.0
4,BLAC_SimAnneal-ET_Global_Init_1-0024-00-3e7216...,2742.0,1929.0,0.838183,"[0.71883655, 0.8054126, 0.8365047, 1.0402751, ...",MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,BLAC,NaN,ANLLLTTIGGPAELTVFTHNMGDHVTRLDSWNPELNEAIPNDTRDT...,True,True,None,CGCGTCGAGTAGGGAAGACAATGCGGCCAACTTACTTCTGACAACG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,7.0,293.0


(24137, 17)


Drop unsuccessful cloning events. Should drop < 150 (146 as of the latest run).

In [4]:
successful_clone_df = master_df.dropna(subset=['dna_to_order'])
print(successful_clone_df.shape)

(23991, 17)


In [5]:
twist_df = successful_clone_df[['id', 'dna_to_order']]
twist_df.head()

,id,dna_to_order
0,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...
1,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...
2,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...
3,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...
4,GFP_SimAnneal-ET_Global_Init_1-0024-00-3e72164...,GGGTCACGCGTAGGGGTCTCACGTGAGCGGCGAGGGTGAAGGTGAC...


## Export

In [6]:
output_file = os.path.join(data_io_utils.S3_DATA_ROOT, 'chip_1', 'twist_order.csv')

twist_df.to_csv(output_file, index=False)

print(data_io_utils.generate_md5_checksum(output_file))

56cc331dd8b824dc19b2783404509a8b


In [7]:
expected_md5 = '56cc331dd8b824dc19b2783404509a8b'

assert data_io_utils.generate_md5_checksum(output_file) == expected_md5

# Sync only if MD5 matches.
# Post publication note: Commenting out sync to our private bucket. 
#data_io_utils.sync_local_path_to_s3(output_file, is_single_file=True)